<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/post_ollama_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import requests
import json
import os
import time

In [103]:
#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [104]:
#un thread demone avvia il server locale di ollama
import subprocess
import threading
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
t.start()

In [105]:
#un altro thread demone avvia llama3
!ollama pull llama3
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3"]),daemon=True)
t2.start()

2024/06/30 12:58:48 routes.go:1064: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-06-30T12:58:48.674Z level=INFO source=images.go:730 msg="total blobs: 5"
time=2024-06-30T12:58:48.674Z level=INFO source=images.go:737 msg="total unused blobs removed: 0"
time=2024-0

[GIN] 2024/06/30 - 12:58:53 | 200 |      53.788µs |       127.0.0.1 | HEAD     "/"


time=2024-06-30T12:58:53.655Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60101]"
time=2024-06-30T12:58:53.773Z level=INFO source=types.go:98 msg="inference compute" id=GPU-7ea08e54-55f5-0453-4813-b05fa6628449 library=cuda compute=6.0 driver=12.2 name="Tesla P100-PCIE-16GB" total="15.9 GiB" available="15.6 GiB"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ [GIN] 2024/06/30 - 12:58:54 | 200 |  969.241606ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [106]:
moderator_instructions = [
    "You are a moderator. Given the following rules, please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n",
    "You are a owner of the Social. Given the following rules,please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n",
    "You are a dictator moderator. Given the following rules, please provide a score from 1 to 10 based on how well the rules are respected, and don't give a brief explanation of your score:\n"
]


def ask_to_llama(instruction,rules, prompt):    
    full_context = instruction
    for i, rule in enumerate(rules, start=1):
        full_context += f"{i}. {rule['text']}\n"
    
    full_prompt = full_context + prompt
    
    response = requests.post('http://localhost:11434/api/generate', 
                             data=json.dumps({'model': 'llama3', 'prompt': full_prompt, 'stream': False}), 
                             headers={'Content-Type': 'application/json'})
    
    return response.json()['response']

In [107]:
def process_json_files(instances):
    for filename in os.listdir(instances):
        if filename.endswith('.json'):
            filepath = os.path.join(instances, filename)
            with open(filepath, 'r') as file:
                data = json.load(file)
                
                rules = data['rules']
                
                records = data['records']
                print(data['name']," con post: ",len(records))
                #if(len(records)>40000):
                    i=0
                    start_time = time.time()
                    for record in data['records']:
                        record_text = record['text']
                        #print(record_text)
                        i+=1
                        for instruction in moderator_instructions:
                            response = ask_to_llama(instruction,rules,record_text)
                            print(f"Response for record ID {record['id']} with instruction '{instruction.split()[3]}': {response}")
                        if (i%100 == 0):
                            tempo = time.time() - start_time
                            print(f"PROCESSATE {i} in tempo: ",tempo)
        break
    end_time = time.time() 
    duration = end_time - start_time
    print("DURATA: ", duration)

[GIN] 2024/06/30 - 12:58:54 | 200 |      32.268µs |       127.0.0.1 | HEAD     "/"


In [108]:
instances = '/kaggle/input/instance-json/results'
process_json_files(instances)

[GIN] 2024/06/30 - 12:58:54 | 200 |   29.423053ms |       127.0.0.1 | POST     "/api/show"
INFO [main] build info | build=1 commit="7c26775" tid="137490889031680" timestamp=1719752335
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="137490889031680" timestamp=1719752335 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="3" port="42003" tid="137490889031680" timestamp=1719752335


⠋ time=2024-06-30T12:58:55.013Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.required.full="5.0 GiB" memory.required.partial="5.0 GiB" memory.required.kv="256.0 MiB" memory.required.allocations="[5.0 GiB]" memory.weights.total="3.9 GiB" memory.weights.repeating="3.5 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="164.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-06-30T12:58:55.014Z level=INFO source=server.go:368 msg="starting llama server" cmd="/tmp/ollama3022589128/runners/cuda_v11/ollama_llama_server --model /root/.ollama/models/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa --ctx-size 2048 --batch-size 512 --embedding --log-disable --n-gpu-layers 33 --parallel 1 --port 42003"
time=2024-06-30T12:58:55.014Z level=INFO source=sched.go:382 msg="loaded runners" count=1
time=2024-06-30T12:58:55.014Z level=INFO source=server

tchafia.be  con post:  50000


   general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q4_0:  225 tensors
llama_model_loader: - type q6_K:    1 tensors
⠦ ⠦ ⠇ ⠏ ⠋ ⠙ llm_load_vocab: special tokens cache size = 256
⠹ ⠸ ⠼ llm_load_vocab: token to piece cache size = 0.8000 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_pr

INFO [main] model loaded | tid="137490889031680" timestamp=1719752338
[GIN] 2024/06/30 - 12:58:58 | 200 |  3.556018035s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:00 | 200 |  4.947356516s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:02 | 200 |  2.187857166s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:04 | 200 |  2.148317741s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:07 | 200 |  2.183738435s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:09 | 200 |  2.689820941s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:12 | 200 |  2.181021805s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:14 | 200 |  2.183889066s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:16 | 200 |  2.013078614s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/06/30 - 12:59:19 | 200 |  3.153835857s |       127.0.0.1 | POST 

KeyboardInterrupt: 